# 🚀 Install, Import, and Log In

## Install libraries

In [ ]:
!pip install -q speechbrain --upgrade -qqq
!pip install wandb --upgrade -qqq
!pip install torchmetrics --upgrade -qqq
!pip install torchinfo --upgrade -qqq
!pip install onnx --upgrade -qqq
!pip install transformers -qqq
!pip install einops --upgrade -qqq
!pip install nlpaug --upgrade -qqq
!pip install pytorch-metric-learning --upgrade -qqq
!pip install git+https://github.com/huggingface/accelerate -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.3/555.3 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 71.1 MB/s eta 

## Weights and Biases login

In [ ]:
import wandb
wandb.login() # Log in with your wandb creditentials

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

from torchmetrics import Accuracy
from torchinfo import summary

import torchaudio
import nlpaug.augmenter.audio as naa
import nlpaug.flow as naf

from speechbrain.nnet.losses import AdditiveAngularMargin, LogSoftmaxWrapper
from speechbrain.lobes.models.ECAPA_TDNN import AttentiveStatisticsPooling

from pytorch_metric_learning import losses, samplers

import onnx
from onnx import shape_inference

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import math
from tqdm import tqdm
import random
import pickle

from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder

from einops import rearrange

from transformers import get_cosine_with_hard_restarts_schedule_with_warmup

from accelerate import Accelerator

import warnings
warnings.filterwarnings("ignore")

## Set-up better GPU stack trace

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

## Ensure deterministic behavior

In [ ]:
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

## Set up GPU

In [ ]:
accelerator = Accelerator(mixed_precision = 'fp16')
device = accelerator.device

# 👩‍🔬 Define the Experiment

## Sweep Configuration

In [ ]:
sweep_config = {
    'method': 'grid'
    }

## Sweep Parameters

In [ ]:
parameters_dict = {
    'fold': {
        # 'values': [('01','02'), ('03','04'), ('05','06'), ('07','08'), ('09','10'), ('11','12'), ('13','14'), ('15','16'), ('17','18'), ('19','20'), ('21','22'), ('23','24')] # RAVDESS
        'values': [["03"], ["08"], ["09"], ["10"], ["11"], ["12"], ["13"], ["14"], ["15"], ["16"]]
    }
  }

sweep_config['parameters'] = parameters_dict

## Training General Hyperparameters

In [ ]:
parameters_dict.update({
    'epochs':{
      'value': 40
    },
    'batch_size':{
        'value': 32
    },
    'learning_rate':{
        'value': 5e-3
    },
    'weight_decay':{
        'value': 9e-3
    },
    'warmup_steps':{
        'value': 10
    },
    'num_cycles':{
        'value': 1
    },
    'loss':{
        'value': 'AF'
    },
    'optimizer':{
        'value': 'AdamW'
    },
    'alpha':{
        'value': 1
    },
    'beta':{
        'value': 1
    },
    'temporal_average':{
        'value': 'asp'
    },
    'scale':{
        'value': 1
    },
    'margin':{
        'value': 0.2
    },
    'layer_selection':{
        'value': 'asp'
    },
    'layer_attn_dim':{
        'value': 32
    },
    'time_attn_dim':{
        'value': 256
    },
    'emo_emdb_ratio':{
        'value': 0.5
    },
    'hidden_size':{
        'value': 64
    }
    })

## Speech Features Hyperparameters

In [ ]:
parameters_dict.update({
    'max_length':{
        'value': 350
    },
    'sample_rate':{
        'value': 16000
    },
    'aug_prob':{
        'value': 0.0
    },
    'proj_drop':{
        'value': 0.2
    },
    'attn_drop':{
        'value': 0.0
    }
    })

## Model Specific Hyperparameters

In [ ]:
parameters_dict.update({
    'input_size':{
        'value': (12, 350, 768)
    },
    'embeddings':{
        'value': 768
    },
    'num_heads':{
        'value': 2
    }
    })

## Dataset and input Parameters

In [ ]:
parameters_dict.update({
    'dataset':{
        'value':"EMODB"
    },
    'path':{
        'value':'kntu-asp-dl/EMODB/EMODB:wav2vec_fp16_noft_base'
    },
    'layers':{
        'value': list(range(1,13))
    },
    'classes':{
        'value': 7
    },
    'class_names':{
        'value':['anger', 'anxiety/fear', 'boredom', 'disgust', 'happiness', 'neutral', 'sadness']
    },
    'input_channels':{
        'value': 1
    }
    })

## Metadata about the run

In [ ]:
parameters_dict.update({
    'project_name':{
        'value':"Thesis"
    }
    })

# 🧹 Initialize the Sweep

In [ ]:
save_model = False
sweep_id = input('What is sweep_id? (leave out if this is first sweep) ')
if sweep_id=="":
    sweep_id = wandb.sweep(sweep_config, project=parameters_dict['project_name']['value'])
    save_model = True

What is sweep_id? (leave out if this is first sweep) 
Create sweep with ID: e7czjb2r
Sweep URL: https://wandb.ai/halflingwizard/Thesis/sweeps/e7czjb2r


# 🚰 Constructing the Pipeline

## Define pipeline

In [ ]:
def model_pipeline(hyperparameters=None):

    # tell wandb to get started
    with wandb.init(config=hyperparameters):

      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # download dataset
      paths = []
      for i in config.layers:
        dataset = wandb.use_artifact(config.path+f'_t{i}', type='dataset')
        # dataset path
        path = dataset.download()
        paths.append(path)

      # make the model, data, and optimization problem
      model, best_model, train_loader, test_loader, criterion, optimizer, scheduler = make(config, paths)
      try:
        print(summary(model, (config.batch_size, config.input_size[0], config.input_size[1], config.input_size[2])))
      except:
        print('⚠️ Something is wrong with TorchInfo.')

      # and set-up the metric
      metric = Metric(config.classes, config.layers, train_loader.dataset.prior_prob)

      # and use them to train the model
      train(model, train_loader, test_loader, criterion, optimizer, scheduler, metric, config)

      # load the best model from training
      best_model.load_state_dict(torch.load('model.pkl'))

      # and test its final performance
      test(best_model, test_loader, criterion, metric, config)

      pickle_artifact = wandb.Artifact(
          name = f'Model.pkl',
          type="model",
          metadata=dict(config))

      wandb.save("model.pkl")
      pickle_artifact.add_file("model.pkl")
      wandb.log_artifact(pickle_artifact)


      if save_model == True:
        try:
          # Save the model in the exchangeable ONNX format
          onnx_artifact = wandb.Artifact(
              name = f'Model.onnx',
              type="model",
              metadata=dict(config))

          # Params for ONNX
          dummy_input = torch.randn(config.batch_size, config.input_size, device = device).to(torch.float16)
          input_names = [ "input" ]
          output_names = [ "output" ]

          # export model as ONNX file
          torch.onnx.export(best_model,
                            dummy_input,
                            "pytorch_model.onnx",
                            verbose=False,
                            input_names=input_names,
                            output_names=output_names,
                            export_params=True,
                            dynamic_axes={'input' : {0 : 'batch_size'},
                                          'output' : {0 : 'batch_size'}})
          onnx.save(onnx.shape_inference.infer_shapes(onnx.load("pytorch_model.onnx")), "model.onnx")

          wandb.save("model.onnx")

          onnx_artifact.add_file("model.onnx")

          wandb.log_artifact(onnx_artifact)
        except:
          print("⚠️ Couldn't save the model.")

    return model

## Define loss function

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha = 0.8, gamma = 2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):

        #comment out if your model contains a sigmoid or equivalent activation layer
        sig = nn.Sigmoid()
        inputs = sig(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        #first compute binary cross-entropy
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = self.alpha * (1-BCE_EXP)**self.gamma * BCE

        return focal_loss

In [ ]:
class DaviesBouldin(nn.Module):
    def __init__(self):
        super(DaviesBouldin, self).__init__()

    def pytorch_euclidean(self, a, b):
        return torch.sqrt(torch.sum((a-b)**2))

    def forward(self, X, labels):

        n_cluster = len(torch.bincount(labels.int()))
        cluster_k = [X[labels == k] for k in range(n_cluster)]
        centroids = [torch.mean(k, dim = 0) for k in cluster_k]
        variances = [torch.mean(torch.Tensor([self.pytorch_euclidean(p, centroids[i]) for p in k])) for i, k in enumerate(cluster_k)]
        db = []

        for i in range(n_cluster):
            for j in range(n_cluster):
                if j != i:
                    db.append((variances[i] + variances[j]) / self.pytorch_euclidean(centroids[i], centroids[j]))
        if n_cluster == 1:
          db.append(0)

        return(max(db) / n_cluster)

In [ ]:
class CenterLoss(nn.Module):
    """Center loss.

    Reference:
    Wen et al. A Discriminative Feature Learning Approach for Deep Face Recognition. ECCV 2016.

    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """
    def __init__(self, num_classes=10, feat_dim=2, use_gpu=True):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.use_gpu = use_gpu

        if self.use_gpu:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())
        else:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(x, self.centers.t(), beta=1, alpha=-2)

        classes = torch.arange(self.num_classes).long()
        if self.use_gpu: classes = classes.cuda()
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = distmat * mask.float()
        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size

        return loss

In [ ]:
class RingLoss(nn.Module):
    """
    Refer to paper
    Ring loss: Convex Feature Normalization for Face Recognition
    """
    def __init__(self, type='L2', loss_weight=1.0):
        super(RingLoss, self).__init__()
        self.radius = nn.Parameter(torch.Tensor(1)).cuda()
        self.radius.data.fill_(-1)
        self.loss_weight = loss_weight
        self.type = type

    def forward(self, x):
        x = x.pow(2).sum(dim=1).pow(0.5)
        if self.radius.data[0] < 0: # Initialize the radius with the mean feature norm of first iteration
            self.radius.data.fill_(x.mean().item())
        if self.type == 'L1': # Smooth L1 Loss
            loss1 = F.smooth_l1_loss(x, self.radius.expand_as(x)).mul_(self.loss_weight)
            loss2 = F.smooth_l1_loss(self.radius.expand_as(x), x).mul_(self.loss_weight)
            ringloss = loss1 + loss2
        elif self.type == 'auto': # Divide the L2 Loss by the feature's own norm
            diff = x.sub(self.radius.expand_as(x)) / (x.mean().detach().clamp(min=0.5))
            diff_sq = torch.pow(torch.abs(diff), 2).mean()
            ringloss = diff_sq.mul_(self.loss_weight)
        else: # L2 Loss, if not specified
            diff = x.sub(self.radius.expand_as(x))
            diff_sq = torch.pow(torch.abs(diff), 2).mean()
            ringloss = diff_sq.mul_(self.loss_weight)
        return ringloss

In [ ]:
class CrossEntropy(nn.Module):
    def __init__(self, num_classes = 4, feat_dim = 768):
        super(CrossEntropy, self).__init__()
        self.CELoss = nn.CrossEntropyLoss()
        self.RingLoss = RingLoss()

    def forward(self, inputs, embeddings, targets):

        return self.CELoss(inputs, targets) + self.RingLoss(embeddings)

In [ ]:
class AFLoss(nn.Module):
    def __init__(self, margin, scale, num_layers):
        super().__init__()
        self.AAMLoss = nn.ModuleList([
                                      LogSoftmaxWrapper(AdditiveAngularMargin(margin = margin, scale=scale))
                                      for _ in range(num_layers)
                                      ]
                                    )

    def forward(self, inputs, targets):
        targets, ses_sex_emo = targets
        labels = process_labels(targets, self.AAMLoss[0])
        loss = 0
        for i, layer in enumerate(self.AAMLoss):
          loss += layer(inputs[:, i], labels)
        return loss

## Define Metrics

In [ ]:
class Metric:
  def __init__(self, num_classes, layers, prior_probabilities):

    self.UA = Accuracy(task = 'multiclass',
                       num_classes = num_classes,
                       average = 'macro')
    self.prior_prob = prior_probabilities
    self.layers = layers

  def majority_vote(self, outputs):
    sm = nn.Softmax(dim=2)
    outputs = sm(outputs)

    predicted, _ = torch.mode(outputs.argmax(dim=2), dim=1)
    return predicted.squeeze().tolist()

  def fixed_weighted_mean(self, outputs):
    weights = torch.tensor([0.070, 0.072, 0.076, 0.078,
                            0.081, 0.084, 0.083, 0.085,
                            0.082, 0.075, 0.071, 0.078],
                           device = device) # from pepnio et al.
    weights = weights[[id-1 for id in self.layers]]
    sm = torch.nn.Softmax(dim=1)

    w = sm(weights.unsqueeze(0).unsqueeze(2))
    predicted = torch.argmax(torch.sum(outputs*w, dim=1, keepdim=True), dim=2)

    return predicted.squeeze().tolist()

  def max_max(self, outputs):
    sm = nn.Softmax(dim=2)
    outputs = sm(outputs)

    outputs, _ = torch.max(outputs, dim=1)

    predicted = torch.argmax(outputs, dim=1)
    return predicted.squeeze().tolist()

  def distribution_summation(self, outputs):
    sm = nn.Softmax(dim=2)
    outputs = sm(outputs)

    predicted = torch.argmax(outputs.sum(dim=1), dim=1)
    return predicted.squeeze().tolist()

  def dempster_shafer(self, outputs):
    sm = nn.Softmax(dim=2)
    outputs = sm(outputs)

    bpa = 1 - torch.prod((1-outputs), dim=1)
    A = torch.sum(bpa/(1-bpa)+1, dim=1, keepdim=True)
    Bel = (1/A)*(bpa/(1-bpa))
    predicted = torch.argmax(Bel, dim=1)

    return predicted.squeeze().tolist()

  def naive_bayes(self, outputs):
    sm = nn.Softmax(dim=2)
    outputs = sm(outputs)
    prior = torch.tensor(self.prior_prob, device = device).unsqueeze(0).unsqueeze(1)

    predicted = torch.argmax(prior*torch.prod(outputs/prior, dim=1, keepdim=True), dim=2)

    return predicted.squeeze().tolist()

  def entropy_weighting(self, outputs):
    sm = nn.Softmax(dim=2)
    outputs = sm(outputs)

    e = -torch.sum(outputs*torch.log(outputs), dim=2, keepdim=True)
    predicted = torch.argmax(torch.sum((outputs==torch.max(outputs, dim=2,keepdim=True)[0]).int()*e, dim=1, keepdim=True), dim=2)

    return predicted.squeeze().tolist()

  def density_based_weighting(self, outputs):
    sm = nn.Softmax(dim=2)
    outputs = sm(outputs)

    predicted = torch.argmax(torch.sum((outputs==torch.max(outputs, dim=2,keepdim=True)[0]).int()*outputs, dim=1, keepdim=True), dim=2)

    return predicted.squeeze().tolist()

  def entropy_weighted_mean(self, outputs):
    sm = nn.Softmax(dim=2)
    outputs = sm(outputs) #p

    entropy = torch.sum(outputs * torch.log(1/outputs), dim=2, keepdim=True)
    weight = 1/entropy
    outputs = outputs * weight

    predicted = torch.argmax(outputs.sum(dim=1), dim=1)
    return predicted.squeeze().tolist()

  def calculate_acc(self, predictions, labels):
    y_true = labels[0].tolist()
    y_pred = predictions

    return self.UA(torch.tensor(y_pred), torch.tensor(y_true))

## Define `make`

In [ ]:
def make(config,  paths):
    # Make the data
    train, test =  get_data(config,  paths, train=True), get_data(config,  paths, train=False)
    train_loader = make_loader(train, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)

    # Make the model
    model = Model(config)

    # Make best model
    best_model = Model(config)

    # Make the loss and optimizer
    criterion = get_loss(config.loss, config)
    params = list(model.parameters()) + list(criterion.parameters())

    optimizer = torch.optim.AdamW(params,
                                  lr = config.learning_rate,
                                  weight_decay = config.weight_decay)

    # Make the scheduler
    scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer,
                                                                   num_warmup_steps = config.warmup_steps,
                                                                   num_training_steps = config.epochs,
                                                                   num_cycles = config.num_cycles)

    model, best_model, optimizer, train_loader, test_loader = accelerator.prepare(model, best_model, optimizer, train_loader, test_loader)

    return model, best_model, train_loader, test_loader, criterion, optimizer, scheduler

In [ ]:
def get_loss(loss_function, config):
  if loss_function.startswith('CE'):
    loss = CrossEntropy(config.classes, config.embeddings)
  elif loss_function.startswith("MM"):
    loss = nn.MultiMarginLoss()
  elif loss_function.startswith("MLSM"):
    loss = nn.MultiLabelSoftMarginLoss()
  elif loss_function.startswith("FL"):
    loss = FocalLoss(alpha=0.75, gamma = 2)
  elif loss_function.startswith("BCE"):
    loss = nn.BCEWithLogitsLoss()
  elif loss_function.startswith("AF"):
    loss = AFLoss(margin = config.margin, scale=config.scale, num_layers=len(config.layers))
  return loss

# 🔊 Speech Processing

## Reading wav file

In [ ]:
def get_waveform(file_path, sample_rate):
  wf, sr = torchaudio.load(file_path)
  resample = torchaudio.transforms.Resample(sr, sample_rate)
  waveform = resample(wf)
  return waveform

## Data Augmentation

In [ ]:
def augment(waveform, aug_prob, sample_rate):

  aug = naf.Sometimes([
    naa.SpeedAug(factor=(0.8, 1.2)),
    naa.LoudnessAug(),
    naa.VtlpAug(sample_rate)
    ],
    aug_p = aug_prob)

  waveform = waveform.numpy()
  waveform = waveform.reshape(-1)
  waveform = aug.augment(waveform)
  waveform = torch.from_numpy(waveform)
  return torch.unsqueeze(waveform, 0)

## Tile waveform

In [ ]:
def tile(waveform, expected_time):
  waveform_time = waveform.shape[1]
  expected_time = expected_time
  repeat_times = (expected_time // waveform_time) + 1
  tiled_data = waveform.repeat(1, repeat_times)
  return tiled_data[:, :expected_time]

## Normalize

In [ ]:
def standardize(waveform):
  means = waveform.mean()
  stds = waveform.std()
  waveform = (waveform - means) / (stds + 1e-7)
  return waveform

# 📡 Data Loading

## Data loader

In [ ]:
class get_data(Dataset):
  def __init__(self, config,  paths, train):
    self.speaker_out = [f"{int(x):02}" for x in config.fold]
    self.train = train

    self.paths =  paths
    self.files = self.filter_files(self.paths[0])

    self.length = config.max_length
    self.sample_rate = config.sample_rate

    self.labels_list = [self.get_label(file_name)[1] for file_name in self.files]
    print(self.labels_list[:10])
    self.num_classes = config.classes
    self.majority_count = max([sum([item==label for item in self.labels_list]) for label in set(self.labels_list)])
    self.prior_prob = [self.labels_list.count(label)/len(self.labels_list) for label in range(self.num_classes)]

    self.aug_prob = config.aug_prob

  def filter_files(self, path):
    list_of_files = os.listdir(path)
    if not self.train:
      file_list = [item[:-3] for item in list_of_files if item.startswith(self.speaker_out[0])]
      return file_list
    elif self.train:
      file_list = [item[:-3] for item in list_of_files if not(item.startswith(self.speaker_out[0]))]
      return file_list

  def get_wav(self, file):
    waveform = get_waveform(file, self.sample_rate)
    if waveform.shape[0] > 1:
      waveform = torch.mean(waveform, dim=0).unsqueeze(0)
    if self.train:
      waveform = augment(waveform, self.aug_prob, self.sample_rate)
    tiled_wav = tile(waveform, self.length)
    std_wav = standardize(tiled_wav)
    return std_wav

  def get_features(self, path, durations):
    short_x = torch.load(path)
    x = []
    for f_i, d_i in zip(short_x, durations):
      x += [float(f_i)] * int(d_i)
    x = torch.FloatTensor(x)
    x = torch.unsqueeze(x, 0)
    x = standardize(x)
    x = tile(x, self.length)
    return x

  def get_label(self, file_name):
    emo_dict = {
    'W':0, # anger
    'A':1, # anxiety/fear
    'L':2, # boredom
    'E':3, # disgust
    'F':4, # happiness
    'T':5, # sadness
    'N':6, # neutral
    }

    gender_dict = {
      '03': 0,
      '08': 1,
      '09': 1,
      '10': 0,
      '11': 0,
      '12': 0,
      '13': 1,
      '14': 1,
      '15': 0,
      '16': 1
    }
    # emotion_char = file_name[6:8] #RAVDESS
    emotion_char = file_name[5] # EMODB
    # emotion_char = file_name[19] #IEMOCAP
    # session_char = file_name[4]
    # gender_char = file_name[15]
    fold_char = file_name[:2]
    label = int(int(fold_char) * 100 + int(gender_dict[fold_char]) * 10 + emo_dict[emotion_char])
    return (emo_dict[emotion_char], label)


  def __len__(self):
    return len(self.files)

  def __getitem__(self, index):
    file_name = self.files[index]

    features = []

    for path in self.paths:
      file_path = os.path.join(path, file_name + '.pt')
      e = torch.load(file_path)
      e = torch.squeeze(e)
      e = rearrange(e, "l c -> c l")
      e = tile(e, self.length)
      e = rearrange(e, "c l -> l c")
      features.append(e)

    features = torch.stack(features, dim=0)

    label = self.get_label(file_name)
    return features, label

In [ ]:
def make_loader(dataset, batch_size):
  if dataset.train:
    sampler = samplers.MPerClassSampler(labels=dataset.labels_list, m=batch_size//dataset.num_classes, batch_size=batch_size, length_before_new_iter=dataset.majority_count*dataset.num_classes)
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size,
                                         sampler = sampler)
  else:
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size)
  return loader

# 🏗️ Build the model

## Attentive Stats Pooling

In [ ]:
class ASP(nn.Module):

    def __init__(self, num_emdb, attn_dim=None):
        super().__init__()
        if not attn_dim:
          attn_dim = num_emdb
        self.asp = AttentiveStatisticsPooling(channels=num_emdb, attention_channels=attn_dim, global_context=True)

    def forward(self, x: torch.Tensor):

        x = rearrange(x, "b l c -> b c l")
        x = self.asp(x)
        x = x.squeeze()

        x, _ = torch.split(x, 768, dim=1)

        return x

## Maxout

In [ ]:
class Maxout(nn.Module):

    def __init__(self, d_in, d_out, pool_size):
        super().__init__()
        self.d_in, self.d_out, self.pool_size = d_in, d_out, pool_size
        self.lin = nn.Linear(d_in, d_out * pool_size)


    def forward(self, inputs):
        shape = list(inputs.size())
        shape[-1] = self.d_out
        shape.append(self.pool_size)
        max_dim = len(shape) - 1
        out = self.lin(inputs)
        m, i = out.view(*shape).max(max_dim)
        return m

## Classification Head

In [ ]:
class ClassificationHead(nn.Module):

    def __init__(self, num_emdb, num_class, p=0.1):
        super().__init__()

        self.norm = nn.LayerNorm(normalized_shape = num_emdb)
        self.dropout = nn.Dropout(p=p)
        self.head = nn.Linear(num_emdb, num_class)


    def forward(self, x):

        x = self.head(self.dropout(self.norm(x)))
        return x

## Model

In [ ]:
class Model(nn.Module):

    def __init__(self, config):
        super().__init__()

        self.experts = nn.ModuleList([
                                    nn.Sequential(
                                        ASP(num_emdb = config.embeddings, attn_dim=config.time_attn_dim),
                                        nn.Dropout(p=0.1),
                                        Maxout(d_in = config.embeddings, d_out = config.hidden_size, pool_size = 3),
                                        ClassificationHead(num_emdb= config.hidden_size, num_class= config.classes, p= config.proj_drop)
                                    )
                                    for _ in range(len(config.layers))
                                    ]
                                  )

    def forward(self, x: torch.Tensor, extra_logs=False):

        all_results = []
        for i, expert in enumerate(self.experts):
          all_results.append(expert(x[:,i]))
        x = torch.stack(all_results, dim=1)

        return x

# 👟 Train the model

In [ ]:
def train(model, train_loader, dev_loader, criterion, optimizer, scheduler, metric, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=1)

    best_acc   = 0.0

    for epoch in tqdm(range(config.epochs)):

      train_loss = []
      train_acc_majority_vote  = []
      train_acc_fixed_weighted_mean = []
      train_acc_max_max = []
      train_acc_distribution_summation = []
      train_acc_dempster_shafer = []
      train_acc_naive_bayes = []
      train_acc_entropy_weighting = []
      train_acc_density_based_weighting = []
      train_acc_entropy_weighted_mean = []
      dev_loss   = []
      dev_acc_majority_vote  = []
      dev_acc_fixed_weighted_mean = []
      dev_acc_max_max = []
      dev_acc_distribution_summation = []
      dev_acc_dempster_shafer = []
      dev_acc_naive_bayes = []
      dev_acc_entropy_weighting = []
      dev_acc_density_based_weighting = []
      dev_acc_entropy_weighted_mean = []

      for inputs, labels in train_loader:
          loss, accuracies = train_batch(inputs, labels, model, optimizer, criterion, metric, config)
          train_loss.append(loss.item())
          train_acc_majority_vote.append(accuracies['majority_vote'].item())
          train_acc_fixed_weighted_mean.append(accuracies['fixed_weighted_mean'].item())
          train_acc_max_max.append(accuracies['max_max'].item())
          train_acc_distribution_summation.append(accuracies['distribution_summation'].item())
          train_acc_dempster_shafer.append(accuracies['dempster_shafer'].item())
          train_acc_naive_bayes.append(accuracies['naive_bayes'].item())
          train_acc_entropy_weighting.append(accuracies['entropy_weighting'].item())
          train_acc_density_based_weighting.append(accuracies['density_based_weighting'].item())
          train_acc_entropy_weighted_mean.append(accuracies['entropy_weighted_mean'].item())

      for inputs, labels in dev_loader:

          loss, accuracies = dev_batch(inputs, labels, model, criterion, metric, config)
          dev_loss.append(loss.item())
          dev_acc_majority_vote.append(accuracies['majority_vote'].item())
          dev_acc_fixed_weighted_mean.append(accuracies['fixed_weighted_mean'].item())
          dev_acc_max_max.append(accuracies['max_max'].item())
          dev_acc_distribution_summation.append(accuracies['distribution_summation'].item())
          dev_acc_dempster_shafer.append(accuracies['dempster_shafer'].item())
          dev_acc_naive_bayes.append(accuracies['naive_bayes'].item())
          dev_acc_entropy_weighting.append(accuracies['entropy_weighting'].item())
          dev_acc_density_based_weighting.append(accuracies['density_based_weighting'].item())
          dev_acc_entropy_weighted_mean.append(accuracies['entropy_weighted_mean'].item())

      scheduler.step()

      # Report metrics every epoch
      wandb.log({
            "Train loss": np.mean(train_loss),
            "Train Accuracy (Majority Vote)": np.mean(train_acc_majority_vote),
            "Train Accuracy (Fixed-weighted Mean)": np.mean(train_acc_fixed_weighted_mean),
            "Train Accuracy (Max of Max)": np.mean(train_acc_max_max),
            "Train Accuracy (Distribution Summation)": np.mean(train_acc_distribution_summation),
            "Train Accuracy (Dempster Shafer)": np.mean(train_acc_dempster_shafer),
            "Train Accuracy (Naive Bayes)": np.mean(train_acc_naive_bayes),
            "Train Accuracy (Entropy Weighting)": np.mean(train_acc_entropy_weighting),
            "Train Accuracy (Density based Weighting)": np.mean(train_acc_density_based_weighting),
            "Train Accuracy (Entropy-weighted Mean)": np.mean(train_acc_entropy_weighted_mean),
            "Validation loss": np.mean(dev_loss),
            "Validation Accuracy (Majority Vote)": np.mean(dev_acc_majority_vote),
            "Validation Accuracy (Fixed-weighted Mean)": np.mean(dev_acc_fixed_weighted_mean),
            "Validation Accuracy (Max of Max)": np.mean(dev_acc_max_max),
            "Validation Accuracy (Distribution Summation)": np.mean(dev_acc_distribution_summation),
            "Validation Accuracy (Dempster Shafer)": np.mean(dev_acc_dempster_shafer),
            "Validation Accuracy (Naive Bayes)": np.mean(dev_acc_naive_bayes),
            "Validation Accuracy (Entropy Weighting)": np.mean(dev_acc_entropy_weighting),
            "Validation Accuracy (Density based Weighting)": np.mean(dev_acc_density_based_weighting),
            "Validation Accuracy (Entropy-weighted Mean)": np.mean(dev_acc_entropy_weighted_mean)
            },
          step=epoch)

      # Keep best model
      new_acc = np.mean([
          np.mean(dev_acc_majority_vote),
          np.mean(dev_acc_fixed_weighted_mean),
          np.mean(dev_acc_max_max),
          np.mean(dev_acc_distribution_summation),
          np.mean(dev_acc_dempster_shafer),
          np.mean(dev_acc_naive_bayes),
          np.mean(dev_acc_entropy_weighting),
          np.mean(dev_acc_density_based_weighting),
          np.mean(dev_acc_entropy_weighted_mean)
      ])
      if new_acc > best_acc:
        best_acc = max(best_acc, new_acc)
        torch.save(model.state_dict(), 'model.pkl')

In [ ]:
def train_batch(inputs, labels, model, optimizer, criterion, metric, config):

    model = model.train()

    # ensuring that the model is in train mode.
    model.train()

    # Forward pass ➡
    outputs = model(inputs)
    targets = process_labels(labels, criterion)
    loss = criterion(outputs, targets)

    # Backward pass ⬅
    optimizer.zero_grad()
    accelerator.backward(loss)

    # Step with optimizer
    optimizer.step()

    # Calculate accuracy
    accuracies = {}
    accuracies['majority_vote'] = metric.calculate_acc(metric.majority_vote(outputs), labels)
    accuracies['fixed_weighted_mean'] = metric.calculate_acc(metric.fixed_weighted_mean(outputs), labels)
    accuracies['max_max'] = metric.calculate_acc(metric.max_max(outputs), labels)
    accuracies['distribution_summation'] = metric.calculate_acc(metric.distribution_summation(outputs), labels)
    accuracies['dempster_shafer'] = metric.calculate_acc(metric.dempster_shafer(outputs), labels)
    accuracies['naive_bayes'] = metric.calculate_acc(metric.naive_bayes(outputs), labels)
    accuracies['entropy_weighting'] = metric.calculate_acc(metric.entropy_weighting(outputs), labels)
    accuracies['density_based_weighting'] = metric.calculate_acc(metric.density_based_weighting(outputs), labels)
    accuracies['entropy_weighted_mean'] = metric.calculate_acc(metric.entropy_weighted_mean(outputs), labels)

    return loss, accuracies

In [ ]:
def dev_batch(inputs, labels, model, criterion, metric, config):
    model = model.train()

    # ensuring that the model is in inference mode.
    model.eval()

    # Forward pass ➡
    outputs = model(inputs)
    targets = process_labels(labels, criterion)
    loss = criterion(outputs, targets)

    # Calculate accuracy
    accuracies = {}
    accuracies['majority_vote'] = metric.calculate_acc(metric.majority_vote(outputs), labels)
    accuracies['fixed_weighted_mean'] = metric.calculate_acc(metric.fixed_weighted_mean(outputs), labels)
    accuracies['max_max'] = metric.calculate_acc(metric.max_max(outputs), labels)
    accuracies['distribution_summation'] = metric.calculate_acc(metric.distribution_summation(outputs), labels)
    accuracies['dempster_shafer'] = metric.calculate_acc(metric.dempster_shafer(outputs), labels)
    accuracies['naive_bayes'] = metric.calculate_acc(metric.naive_bayes(outputs), labels)
    accuracies['entropy_weighting'] = metric.calculate_acc(metric.entropy_weighting(outputs), labels)
    accuracies['density_based_weighting'] = metric.calculate_acc(metric.density_based_weighting(outputs), labels)
    accuracies['entropy_weighted_mean'] = metric.calculate_acc(metric.entropy_weighted_mean(outputs), labels)

    return loss, accuracies

In [ ]:
def process_labels(labels, criterion):
  targets = labels
  if str(criterion) in ['MultiLabelSoftMarginLoss()', 'FocalLoss()', 'BCEWithLogitsLoss()', 'AdditiveAngularMargin()']:
    enc = OneHotEncoder()
    enc.fit(np.array([0,1,2,3,4,5,6]).reshape(-1, 1))
    cpu_labels = labels.cpu()
    targets = torch.tensor(enc.transform(cpu_labels.reshape(-1, 1)).todense(), dtype=torch.float, device = device)
  elif str(criterion) in ['LogSoftmaxWrapper(\n  (loss_fn): AdditiveAngularMargin()\n  (criterion): KLDivLoss()\n)']:
    targets = targets.unsqueeze(1)
  return targets

# 🧪 Test and Evaluation

In [ ]:
def test(model, test_loader, criterion, metric, config):
  y_pred = []
  pred_majority_vote  = []
  pred_fixed_weighted_mean = []
  pred_max_max = []
  pred_distribution_summation = []
  pred_dempster_shafer = []
  pred_naive_bayes = []
  pred_entropy_weighting = []
  pred_density_based_weighting = []
  pred_entropy_weighted_mean = []
  y_true = []
  expert_predictions = []

  # ensuring that the model is in inference mode.
  model = model.train()
  model.eval()

  # Run the model on some test examples
  for inputs, labels in test_loader:
      outputs = model(inputs, extra_logs=True)

      y_true += labels[0].tolist()

      pred_majority_vote += metric.majority_vote(outputs)
      pred_fixed_weighted_mean += metric.fixed_weighted_mean(outputs)
      pred_max_max += metric.max_max(outputs)
      pred_distribution_summation += metric.distribution_summation(outputs)
      pred_dempster_shafer += metric.dempster_shafer(outputs)
      pred_naive_bayes += metric.naive_bayes(outputs)
      pred_entropy_weighting += metric.entropy_weighting(outputs)
      pred_density_based_weighting += metric.density_based_weighting(outputs)
      pred_entropy_weighted_mean += metric.entropy_weighted_mean(outputs)

      expert_predictions += outputs.argmax(dim=2).tolist()

  # weighted and unweighted accuracy

  WA = Accuracy(task = 'multiclass',
                num_classes = config.classes,
                average = 'micro')
  UA = Accuracy(task = 'multiclass',
                num_classes = config.classes,
                average = 'macro')

  # selected layers table
  methods = ['Majority Vote' ,'Fixed-weighted Mean' ,'Max of Max' ,'Distribution Summation' ,'Dempster Shafer' ,'Naive Bayes' ,'Entropy Weighting' ,'Density based Weighting' ,'Entropy-weighted Mean']
  y_pred += [list(x) for x in zip(
      pred_majority_vote,
      pred_fixed_weighted_mean,
      pred_max_max,
      pred_distribution_summation,
      pred_dempster_shafer,
      pred_naive_bayes,
      pred_entropy_weighting,
      pred_density_based_weighting,
      pred_entropy_weighted_mean,
      y_true
  )]
  my_data = [a + b for a, b in list(zip(expert_predictions, y_pred))]
  columns=[f'Expert #{i+1} predictions' for i in range(len(config.layers))]+methods+['Target']


  wandb.log({
      "Weighted Accuracy (Majority Vote)": WA(torch.tensor(pred_majority_vote), torch.tensor(y_true)),
      "Unweighted Accuracy (Majority Vote)": UA(torch.tensor(pred_majority_vote), torch.tensor(y_true)),
      "Weighted Accuracy (Fixed-weighted Mean)": WA(torch.tensor(pred_fixed_weighted_mean), torch.tensor(y_true)),
      "Unweighted Accuracy (Fixed-weighted Mean)": UA(torch.tensor(pred_fixed_weighted_mean), torch.tensor(y_true)),
      "Weighted Accuracy (Max of Max)": WA(torch.tensor(pred_max_max), torch.tensor(y_true)),
      "Unweighted Accuracy (Max of Max)": UA(torch.tensor(pred_max_max), torch.tensor(y_true)),
      "Weighted Accuracy (Distribution Summation)": WA(torch.tensor(pred_distribution_summation), torch.tensor(y_true)),
      "Unweighted Accuracy (Distribution Summation)": UA(torch.tensor(pred_distribution_summation), torch.tensor(y_true)),
      "Weighted Accuracy (Dempster Shafer)": WA(torch.tensor(pred_dempster_shafer), torch.tensor(y_true)),
      "Unweighted Accuracy (Dempster Shafer)": UA(torch.tensor(pred_dempster_shafer), torch.tensor(y_true)),
      "Weighted Accuracy (Naive Bayes)": WA(torch.tensor(pred_naive_bayes), torch.tensor(y_true)),
      "Unweighted Accuracy (Naive Bayes)": UA(torch.tensor(pred_naive_bayes), torch.tensor(y_true)),
      "Weighted Accuracy (Entropy Weighting)": WA(torch.tensor(pred_entropy_weighting), torch.tensor(y_true)),
      "Unweighted Accuracy (Entropy Weighting)": UA(torch.tensor(pred_entropy_weighting), torch.tensor(y_true)),
      "Weighted Accuracy (Density based Weighting)": WA(torch.tensor(pred_density_based_weighting), torch.tensor(y_true)),
      "Unweighted Accuracy (Density based Weighting)": UA(torch.tensor(pred_density_based_weighting), torch.tensor(y_true)),
      "Weighted Accuracy (Entropy-weighted Mean)": WA(torch.tensor(pred_entropy_weighted_mean), torch.tensor(y_true)),
      "Unweighted Accuracy (Entropy-weighted Mean)": UA(torch.tensor(pred_entropy_weighted_mean), torch.tensor(y_true)),
      "Confusion Matrix (Majority Vote)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_majority_vote, class_names=config.class_names),
      "Confusion Matrix (Fixed-weighted Mean)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_fixed_weighted_mean, class_names=config.class_names),
      "Confusion Matrix (Max of Max)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_max_max, class_names=config.class_names),
      "Confusion Matrix (Distribution Summation)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_distribution_summation, class_names=config.class_names),
      "Confusion Matrix (Dempster Shafer)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_dempster_shafer, class_names=config.class_names),
      "Confusion Matrix (Naive Bayes)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_naive_bayes, class_names=config.class_names),
      "Confusion Matrix (Entropy Weighting)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_entropy_weighting, class_names=config.class_names),
      "Confusion Matrix (Density based Weighting)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_density_based_weighting, class_names=config.class_names),
      "Confusion Matrix (Entropy-weighted Mean)" : wandb.plot.confusion_matrix(probs=None, y_true=y_true, preds=pred_entropy_weighted_mean, class_names=config.class_names),
      "Selected Layers": wandb.Table(data=my_data, columns=columns)
      })

In [ ]:
def readable_targets(labels):
  emo_dict = {
      0:'anger',
      1:'anxiety/fear',
      2:'boredom',
      3:'disgust',
      4:'happiness',
      5:'sadness',
      6:'neutral',
  }
  str_labels = [emo_dict[x] for x in labels]

  return str_labels

# 🏃‍♀️ Run training

In [ ]:
# Build, train and analyze the model with the pipeline
wandb.agent(sweep_id,project=parameters_dict['project_name']['value'],entity='halflingwizard',function=model_pipeline)

wandb: Agent Starting Run: cl1fzebk with config:
wandb: 	alpha: 1
wandb: 	attn_drop: 0
wandb: 	aug_prob: 0
wandb: 	batch_size: 32
wandb: 	beta: 1
wandb: 	class_names: ['anger', 'anxiety/fear', 'boredom', 'disgust', 'happiness', 'neutral', 'sadness']
wandb: 	classes: 7
wandb: 	dataset: EMODB
wandb: 	embeddings: 768
wandb: 	emo_emdb_ratio: 0.5
wandb: 	epochs: 40
wandb: 	fold: ['03']
wandb: 	hidden_size: 64
wandb: 	input_channels: 1
wandb: 	input_size: [12, 350, 768]
wandb: 	layer_attn_dim: 32
wandb: 	layer_selection: asp
wandb: 	layers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
wandb: 	learning_rate: 0.005
wandb: 	loss: AF
wandb: 	margin: 0.2
wandb: 	max_length: 350
wandb: 	num_cycles: 1
wandb: 	num_heads: 2
wandb: 	optimizer: AdamW
wandb: 	path: kntu-asp-dl/EMODB/EMODB:wav2vec_fp16_noft_base
wandb: 	proj_drop: 0.2
wandb: 	project_name: Thesis
wandb: 	sample_rate: 16000
wandb: 	scale: 1
wandb: 	temporal_average: asp
wandb: 	time_attn_dim: 256
wandb: 	warmup_steps: 10
wandb: 	weight_decay: 

wandb: Downloading large artifact EMODB:wav2vec_fp16_noft_base_t1, 108.71MB. 535 files... 
wandb:   535 of 535 files downloaded.  
Done. 0:0:23.3
wandb: Downloading large artifact EMODB:wav2vec_fp16_noft_base_t2, 108.71MB. 535 files... 
wandb:   535 of 535 files downloaded.  
Done. 0:0:21.0
wandb: Downloading large artifact EMODB:wav2vec_fp16_noft_base_t3, 108.71MB. 535 files... 
wandb:   535 of 535 files downloaded.  
Done. 0:0:21.5
wandb: Downloading large artifact EMODB:wav2vec_fp16_noft_base_t4, 108.71MB. 535 files... 
wandb:   535 of 535 files downloaded.  
Done. 0:0:20.8
wandb: Downloading large artifact EMODB:wav2vec_fp16_noft_base_t5, 108.71MB. 535 files... 
wandb:   535 of 535 files downloaded.  
Done. 0:0:20.8
wandb: Downloading large artifact EMODB:wav2vec_fp16_noft_base_t6, 108.71MB. 535 files... 
wandb:   535 of 535 files downloaded.  
Done. 0:0:20.9
wandb: Downloading large artifact EMODB:wav2vec_fp16_noft_base_t7, 108.71MB. 535 files... 
wandb:   535 of 535 files downloa

[913, 913, 1004, 1105, 815, 810, 1206, 1504, 1415, 1501]
[300, 305, 306, 300, 300, 300, 302, 304, 306, 306]
Layer (type:depth-idx)                                  Output Shape              Param #
Model                                                   [32, 12, 7]               --
├─ModuleList: 1-1                                       --                        --
│    └─Sequential: 2-1                                  [32, 7]                   --
│    │    └─ASP: 3-1                                    [32, 768]                 787,968
│    │    └─Dropout: 3-2                                [32, 768]                 --
│    │    └─Maxout: 3-3                                 [32, 64]                  147,648
│    │    └─ClassificationHead: 3-4                     [32, 7]                   583
│    └─Sequential: 2-2                                  [32, 7]                   --
│    │    └─ASP: 3-5                                    [32, 768]                 787,968
│    │    └─Dropout: 

100%|██████████| 40/40 [04:17<00:00,  6.45s/it]


⚠️ Couldn't save the model.


...

[913, 913, 1004, 300, 1105, 815, 810, 1206, 1415, 1316]
[1504, 1501, 1500, 1500, 1506, 1506, 1506, 1500, 1500, 1503]
Layer (type:depth-idx)                                  Output Shape              Param #
Model                                                   [32, 12, 7]               --
├─ModuleList: 1-1                                       --                        --
│    └─Sequential: 2-1                                  [32, 7]                   --
│    │    └─ASP: 3-1                                    [32, 768]                 787,968
│    │    └─Dropout: 3-2                                [32, 768]                 --
│    │    └─Maxout: 3-3                                 [32, 64]                  147,648
│    │    └─ClassificationHead: 3-4                     [32, 7]                   583
│    └─Sequential: 2-2                                  [32, 7]                   --
│    │    └─ASP: 3-5                                    [32, 768]                 787,968
│    │    └─

100%|██████████| 40/40 [04:17<00:00,  6.45s/it]


⚠️ Couldn't save the model.


Train Accuracy (Dempster Shafer),▁▂▂▃▅▅▆▆▆▆▅▆▆▅▇▆▇▇▇▇▆█▇██▇▇█▇▇▇▆▆▆▇█▇▇▇█
Train Accuracy (Density based Weighting),▁▂▂▃▅▅▆▆▆▇▆▆▆▅█▆▇▇▇▇▆█▇██▇▇██▇▇▆▆▆▇█▇▇▇█
Train Accuracy (Distribution Summation),▁▂▂▃▅▅▆▆▆▆▅▆▆▅▇▆▇▇▇▇▆█▇██▇▇█▇▇▇▆▆▆▇█▇▇▇█
Train Accuracy (Entropy Weighting),▁▂▂▃▅▅▆▅▆▆▆▆▆▅█▆▇▇▇▇▆█▇██▇▇██▇▇▆▆▆▇█▇▇▇█
Train Accuracy (Entropy-weighted Mean),▁▂▂▃▅▅▆▅▆▆▅▆▆▅▇▆▇▇▇▇▆█▇▇█▇▇█▇▇▇▆▆▆▇█▇▇▇▇
Train Accuracy (Fixed-weighted Mean),▁▂▂▃▅▅▆▆▆▆▅▆▆▅▇▆▇▇▇▇▆█▇██▇▇█▇▇▇▆▆▆▇█▇▇▇█
Train Accuracy (Majority Vote),▁▂▂▃▅▅▆▆▆▇▆▆▆▅█▆▇▇▇▇▆█▇██▇▇██▇▇▆▆▆▇█▇▇▇█
Train Accuracy (Max of Max),▁▂▂▃▅▄▅▅▆▆▅▆▆▅▇▆▇▇▇▆▆█▆▇█▇▇█▇▇▇▆▆▆▇█▇▇▇█
Train Accuracy (Naive Bayes),█▆▆▆▁███▁█▆▃▆▆█▆▆██▃█▆███▆███▆█▆█▃▆█▃▃█▆
Train loss,██▇▆▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Unweighted Accuracy (Dempster Shafer),▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
